<a href="https://colab.research.google.com/github/MHosseinHashemi/Image_Similarity_CenterLoss_TripletLoss/blob/main/Image_Simmilarity_CenterLoss_TF_v4_last_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, RandomFlip, RandomRotation, Dense, Dropout, Lambda

from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

In [ ]:
(train_data, test_data, validation_data), info = tfds.load("oxford_flowers102", split=['train', 'validation', 'test'], as_supervised=True, with_info=True)

In [ ]:
height = 224
width = 224

def preprocess_images(image, label, height, width):
    # image = tf.image.resize_with_crop_or_pad(image, target_height=height, target_width=width)
    image = tf.image.resize(image, [width, height])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [ ]:
train_ds = train_data.map(lambda image, label: preprocess_images(image, label, height, width))

In [ ]:
test_ds = test_data.map(lambda image, label: preprocess_images(image, label, height, width))

In [ ]:
def data_loader(data):
  x = []
  y = []
  for img, label in tqdm(data.as_numpy_iterator()):
    x.append(img)
    y.append(label)

  return x, y

In [ ]:
x_train, y_train = data_loader(train_ds)

1020it [00:03, 263.87it/s]


In [ ]:
x_test, y_test = data_loader(test_ds)

1020it [00:03, 276.14it/s]


In [ ]:
# Base Model
MODEL_URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2"

input_layer = Input(shape=(height, width, 3))
x = RandomFlip()(input_layer)
x = RandomRotation(0.3)(x)
x = hub.KerasLayer(MODEL_URL, trainable=True)(x) # In the previous version was False
x = Dropout(0.25)(x)
x = Dense(1024, activation=None)(x)
x = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)  # L2 normalize embeddings
output_layer = Dense(102, activation='softmax')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 random_flip (RandomFlip)    (None, 224, 224, 3)       0         
                                                                 
 random_rotation (RandomRot  (None, 224, 224, 3)       0         
 ation)                                                          
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              20331360  
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                             

In [ ]:
def batch_me(images, labels, batch_size, samples_per_class):
  temp_dict = defaultdict(list) # A Dic of Lists to save img, label pairs as one object
  for img, label in zip(images, labels):
    temp_dict[label].append(img)

  while True:
    batch_x = []
    batch_y = []
    while len(batch_x) < batch_size:
      for category, examples in temp_dict.items():
        # Only feed as large as the "samples per class"
        # If the batch did not had enough space, feed as much as it has
        n_samples = min(samples_per_class, (batch_size - len(batch_x)))
        if n_samples == 0:
          break
        # Pick randomly from simmilar images of the same category
        samples = random.sample(examples, k=n_samples)
        # Add corresponding x, y values to the batch
        batch_x.extend(samples)
        batch_y.extend([category] * len(samples))


    # It should be a continous operation
    yield np.array(batch_x), np.array(batch_y)


In [ ]:
def center_loss(feature_vector, center):
    difference = feature_vector - center
    loss = tf.reduce_mean(tf.reduce_sum(difference**2, axis=1))

    return loss

# Feature Extraction

In [ ]:
feature_extraction_model = Model(inputs=model.input, outputs=model.layers[-2].output)

In [ ]:
raw_features = {}
class_feature_vectors = {}  # Store pre-calculated feature vectors for each class

# Calculate and store class feature vectors
for x, y in zip(x_train, y_train):
    if y not in class_feature_vectors:
        feature_vector = feature_extraction_model.predict(np.expand_dims(x, axis=0), verbose = 0).mean()
        class_feature_vectors[y] = feature_vector

# Calculate raw_features using class_feature_vectors
for x, y in zip(x_train, y_train):
    if y in raw_features:
        new_center = feature_extraction_model.predict(np.expand_dims(x, axis=0), verbose = 0).mean()
        raw_features[y] = [(prev + new_center) / 2 for prev in raw_features[y]]
    else:
        raw_features[y] = [class_feature_vectors[y]] * 1024 # 1024 is the output size of feature vectors

all_features = {key: value for key, value in tqdm(raw_features.items())}

# Clean up memory
del class_feature_vectors

100%|██████████| 102/102 [00:00<00:00, 490618.13it/s]


# Training Loop

In [ ]:
epochs = 120
batch_size = 24
n_examples_per_class = 6
EMA_lr = 0.9
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,
#     patience=10,
#     min_lr=1e-6,
#     verbose=1
# )
num_steps_per_epoch = len(x_train) // batch_size

# Calculate initial centers
centers = tf.Variable(initial_value=tf.random.normal((102, 1024), mean=0.0, stddev=0.5))

# Loop through all samples
for index in range(102):
    # category_features = all_features[y_train == index]
    category_center = all_features[y_train == index]
    # if len(category_features) > 0:
    if len(category_center) > 0 :
        # category_center = tf.reduce_mean(category_features, axis=0)
        centers[index].assign(category_center)

# Training
val_losses = []
train_losses = []
for epoch in tqdm(range(epochs)):
    total_loss = 0.0
    num_batches = 0
    train_preds = []
    train_labels = []

    # Schedule the alpha
    if epoch < (epochs)/3:
        alpha = 0.1
    elif (epochs)/3 < epoch < (epochs)*2/3 :
        alpha = 0.25
    else:
        alpha = 0.5

    # Create batch generator for the current epoch
    batch_generator = batch_me(images=x_train, labels=y_train, batch_size=batch_size, samples_per_class=n_examples_per_class)

    for batch_idx in range(num_steps_per_epoch):
        if batch_idx == 0:
            print("\n")
        # Get the next batch
        batch_x, batch_y = next(batch_generator)
        # Capture Gradients
        with tf.GradientTape() as tape:

            # Extract Features per batch
            predictions = model(batch_x)
            # train_preds.extend(predictions.numpy())
            train_preds.extend(predictions)
            train_labels.extend(tf.one_hot(batch_y, 102))

            # initialize batch centers
            batch_centers = centers.numpy()[batch_y]

            # Calculate Batch Centers
            for index in range(batch_size):
                instance_feature = feature_extraction_model(np.expand_dims(batch_x[index], axis=0))
                instance_mean = tf.reduce_mean(instance_feature[0], axis=0)
                batch_centers[index] = [instance_mean] * 1024

            # Center-Loss calculation
            c_loss = center_loss(instance_feature, batch_centers)
            # Combine it with CategoricalCrossEntropyLoss
            cls_loss = tf.keras.losses.CategoricalCrossentropy()(tf.one_hot(batch_y, 102), predictions)
            # Total Loss
            loss = (c_loss * alpha) + cls_loss

        # Calculate Gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Update training loss
        total_loss += loss
        num_batches += 1
        print(f"Epoch: {epoch} -step: {num_batches} -running loss: {loss:.3f}")


    # Calculate training Loss
    training_loss = total_loss / num_batches

    ### DEBUGGIN
    # print(f"train_preds: {train_preds}")
    # print(f"train_preds shape: {train_preds}")
    # print("="*100)
    # print(f"train_preds: {train_labels}")
    # print(f"train_preds shape: {train_labels.shape}")
    ### END OF DEBUGGIN

    """Calculate AUC and ROC for training"""
    # train_auc = roc_auc_score(tf.one_hot(np.argmax(train_labels, axis=1), 102), train_preds, multi_class='ovr')
    # train_auc = tf.keras.metrics.AUC()(tf.one_hot(np.argmax(train_labels, axis=1), 102), train_preds).numpy()
    # train_fpr, train_tpr, _ = roc_curve(tf.one_hot(np.argmax(train_labels, axis=1), 102), train_preds, pos_label=None)
    # train_accuracy = tf.keras.metrics.Accuracy()(np.argmax(train_labels, axis=1), np.argmax(train_preds, axis=1)).numpy()


    """Validation Loop"""
    val_batch_generator = batch_me(images=x_test, labels=y_test, batch_size=batch_size, samples_per_class=n_examples_per_class)
    val_preds = []
    val_labels = []
    for step in range(num_steps_per_epoch):
        val_batch_x, val_batch_y = next(val_batch_generator)
        # Make Predictions
        val_predictions = model(val_batch_x, training=False)
        # val_preds.extend(val_predictions.numpy())
        val_preds.extend(val_predictions)
        val_labels.extend(val_batch_y)
        # Init Centers
        val_batch_centers = centers.numpy()[val_batch_y]


    # Center Calcualtion
    for idx in range(batch_size):
        val_instance_features = feature_extraction_model(np.expand_dims(val_batch_x[idx], axis=0))
        val_instance_mean = tf.reduce_mean(val_instance_features[0], axis=0)
        val_batch_centers[idx] = val_instance_mean * 1024


    # Loss Calculation
    val_c_loss = center_loss(val_instance_features, val_batch_centers)
    val_cls_loss = tf.keras.losses.CategoricalCrossentropy()(tf.one_hot(val_batch_y, 102), val_predictions)
    val_loss = (val_c_loss * alpha) + val_cls_loss
    val_loss = val_loss / num_batches

    # lr_scheduler.on_epoch_end(epoch, logs={'val_loss': val_loss.numpy()})

    """AUC ROC ACC Calculation"""
    val_auc = tf.keras.metrics.AUC()(tf.one_hot(val_labels, 102), val_preds).numpy()
    # val_fpr, val_tpr, _ = roc_curve(tf.one_hot(np.argmax(val_labels, axis=1), 102), val_preds, pos_label=None)
    # val_accuracy = tf.keras.metrics.Accuracy()(np.argmax(val_labels, axis=1), np.argmax(val_preds, axis=1)).numpy()

    print(f"\nEpoch {epoch + 1}/{epochs} - Training Loss: {training_loss:.3f} - Validation Loss: {val_loss.numpy():.3f}")
    val_losses.append(val_loss)
    train_losses.append(training_loss)
    # print(f"Training AUC: {train_auc:.3f} - Validation AUC: {val_auc:.3f}")
    # print(f"Training Accuracy: {train_accuracy:.3f} - Validation Accuracy: {val_accuracy:.3f}")


    # Centers Update Frequency
    for index in range(102):
        category_features = all_features[y_train == index]
        if len(category_features)>0:
            category_center = tf.reduce_mean(category_features, axis=0)
            # centers[index].assign((1.0 - EMA_lr) * centers[index]) + (EMA_lr * category_center)
            centers[index].assign((1.0 - EMA_lr) * tf.cast(centers[index], tf.float32) + (EMA_lr * tf.cast(category_center, tf.float32)))
    print(f"Centers updated - Step : {epoch}")
    print("="*100)

  0%|          | 0/120 [00:00<?, ?it/s]

Epoch: 0 -step: 1 -running loss: 4.729
Epoch: 0 -step: 2 -running loss: 4.386
Epoch: 0 -step: 3 -running loss: 4.166
Epoch: 0 -step: 4 -running loss: 4.085
Epoch: 0 -step: 5 -running loss: 3.930
Epoch: 0 -step: 6 -running loss: 3.793
Epoch: 0 -step: 7 -running loss: 3.686
Epoch: 0 -step: 8 -running loss: 3.653
Epoch: 0 -step: 9 -running loss: 3.487
Epoch: 0 -step: 10 -running loss: 3.437
Epoch: 0 -step: 11 -running loss: 3.325
Epoch: 0 -step: 12 -running loss: 3.261
Epoch: 0 -step: 13 -running loss: 3.180
Epoch: 0 -step: 14 -running loss: 3.129
Epoch: 0 -step: 15 -running loss: 3.073
Epoch: 0 -step: 16 -running loss: 3.024
Epoch: 0 -step: 17 -running loss: 2.983
Epoch: 0 -step: 18 -running loss: 2.938
Epoch: 0 -step: 19 -running loss: 2.896
Epoch: 0 -step: 20 -running loss: 2.856
Epoch: 0 -step: 21 -running loss: 2.815
Epoch: 0 -step: 22 -running loss: 2.774
Epoch: 0 -step: 23 -running loss: 2.731
Epoch: 0 -step: 24 -running loss: 2.690
Epoch: 0 -step: 25 -running loss: 2.649
Epoch: 0 

  1%|          | 1/120 [03:03<6:03:48, 183.44s/it]

Centers updated - Step : 0


Epoch: 1 -step: 1 -running loss: 1.921
Epoch: 1 -step: 2 -running loss: 1.884
Epoch: 1 -step: 3 -running loss: 1.846
Epoch: 1 -step: 4 -running loss: 1.810
Epoch: 1 -step: 5 -running loss: 1.773
Epoch: 1 -step: 6 -running loss: 1.738
Epoch: 1 -step: 7 -running loss: 1.702
Epoch: 1 -step: 8 -running loss: 1.667
Epoch: 1 -step: 9 -running loss: 1.633
Epoch: 1 -step: 10 -running loss: 1.599
Epoch: 1 -step: 11 -running loss: 1.566
Epoch: 1 -step: 12 -running loss: 1.533
Epoch: 1 -step: 13 -running loss: 1.501
Epoch: 1 -step: 14 -running loss: 1.470
Epoch: 1 -step: 15 -running loss: 1.439
Epoch: 1 -step: 16 -running loss: 1.408
Epoch: 1 -step: 17 -running loss: 1.378
Epoch: 1 -step: 18 -running loss: 1.349
Epoch: 1 -step: 19 -running loss: 1.320
Epoch: 1 -step: 20 -running loss: 1.293
Epoch: 1 -step: 21 -running loss: 1.265
Epoch: 1 -step: 22 -running loss: 1.238
Epoch: 1 -step: 23 -running loss: 1.212
Epoch: 1 -step: 24 -running loss: 1.186
Epoch: 1 -step: 25 -

  2%|▏         | 2/120 [04:47<4:29:22, 136.97s/it]

...

Centers updated - Step : 35


Epoch: 36 -step: 1 -running loss: 2.306
Epoch: 36 -step: 2 -running loss: 2.305
Epoch: 36 -step: 3 -running loss: 2.304
Epoch: 36 -step: 4 -running loss: 2.303
Epoch: 36 -step: 5 -running loss: 2.302
Epoch: 36 -step: 6 -running loss: 2.301
Epoch: 36 -step: 7 -running loss: 2.300
Epoch: 36 -step: 8 -running loss: 2.299
Epoch: 36 -step: 9 -running loss: 2.298
Epoch: 36 -step: 10 -running loss: 2.297
Epoch: 36 -step: 11 -running loss: 2.296
Epoch: 36 -step: 12 -running loss: 2.295
Epoch: 36 -step: 13 -running loss: 2.294
Epoch: 36 -step: 14 -running loss: 2.293
Epoch: 36 -step: 15 -running loss: 2.292
Epoch: 36 -step: 16 -running loss: 2.291
Epoch: 36 -step: 17 -running loss: 2.290
Epoch: 36 -step: 18 -running loss: 2.289
Epoch: 36 -step: 19 -running loss: 2.288
Epoch: 36 -step: 20 -running loss: 2.287
Epoch: 36 -step: 21 -running loss: 2.286
Epoch: 36 -step: 22 -running loss: 2.285
Epoch: 36 -step: 23 -running loss: 2.284
Epoch: 36 -step: 24 -running loss: 2

 31%|███       | 37/120 [1:06:08<2:24:30, 104.46s/it]

Centers updated - Step : 36


Epoch: 37 -step: 1 -running loss: 2.264
Epoch: 37 -step: 2 -running loss: 2.263
Epoch: 37 -step: 3 -running loss: 2.262
Epoch: 37 -step: 4 -running loss: 2.261
Epoch: 37 -step: 5 -running loss: 2.260
Epoch: 37 -step: 6 -running loss: 2.259
Epoch: 37 -step: 7 -running loss: 2.258
Epoch: 37 -step: 8 -running loss: 2.257
Epoch: 37 -step: 9 -running loss: 2.256
Epoch: 37 -step: 10 -running loss: 2.255
Epoch: 37 -step: 11 -running loss: 2.254
Epoch: 37 -step: 12 -running loss: 2.253
Epoch: 37 -step: 13 -running loss: 2.252
Epoch: 37 -step: 14 -running loss: 2.251
Epoch: 37 -step: 15 -running loss: 2.250
Epoch: 37 -step: 16 -running loss: 2.249
Epoch: 37 -step: 17 -running loss: 2.248
Epoch: 37 -step: 18 -running loss: 2.247
Epoch: 37 -step: 19 -running loss: 2.246
Epoch: 37 -step: 20 -running loss: 2.245
Epoch: 37 -step: 21 -running loss: 2.245
Epoch: 37 -step: 22 -running loss: 2.244
Epoch: 37 -step: 23 -running loss: 2.243
Epoch: 37 -step: 24 -running loss: 2

 32%|███▏      | 38/120 [1:07:52<2:22:34, 104.32s/it]

Centers updated - Step : 37


Epoch: 38 -step: 1 -running loss: 2.224
Epoch: 38 -step: 2 -running loss: 2.223
Epoch: 38 -step: 3 -running loss: 2.222
Epoch: 38 -step: 4 -running loss: 2.221
Epoch: 38 -step: 5 -running loss: 2.220
Epoch: 38 -step: 6 -running loss: 2.219
Epoch: 38 -step: 7 -running loss: 2.218
Epoch: 38 -step: 8 -running loss: 2.217
Epoch: 38 -step: 9 -running loss: 2.216
Epoch: 38 -step: 10 -running loss: 2.215
Epoch: 38 -step: 11 -running loss: 2.215
Epoch: 38 -step: 12 -running loss: 2.214
Epoch: 38 -step: 13 -running loss: 2.213
Epoch: 38 -step: 14 -running loss: 2.212
Epoch: 38 -step: 15 -running loss: 2.211
Epoch: 38 -step: 16 -running loss: 2.210
Epoch: 38 -step: 17 -running loss: 2.209
Epoch: 38 -step: 18 -running loss: 2.208
Epoch: 38 -step: 19 -running loss: 2.207
Epoch: 38 -step: 20 -running loss: 2.206
Epoch: 38 -step: 21 -running loss: 2.205
Epoch: 38 -step: 22 -running loss: 2.204


In [ ]:
import seaborn as sns
sns.lineplot(x=np.arange(1,121), y = [index.numpy() for index in val_losses])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save_weights("/content/drive/MyDrive/model_v0_120.h5")